In [120]:
"""earthquakes.ipynb"""

# Cell 01

import requests

# Start by making a get request to retreive the data from the URL
# This returns a Response object
# (I looked that up on google)

data = requests.get(
    "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_week.geojson"
)

# Convert the json response to a dictionary
earthquake_dict = data.json()

In [121]:
import numpy as np

# Under the 'features' key of the dictionary, there is a list of the
# earthquakes, with a dictionary associated to each one
# This is an array of those dictionaries
quakes_array = np.array(earthquake_dict["features"])


# To extract data from a single dictionary, call the keys for time,
# place, and magnitude
# They must be preceded with properties, which is a dictionary of the values
def extract_data(a: dict):
    return a["properties"]["time"], a["properties"]["place"], a["properties"]["mag"]


# To extract the data from each entry in the array, I'm vectorizing
# the function. This also makes sure all the data ends up in numpy arrays
f = np.vectorize(extract_data)

# extract_data returns a tuple, so we get a tuple of ndarrays
# I'm going to save each into a corresponding array
time_array, place_array, mag_array = f(quakes_array)

In [122]:
# Now, since we only want to display the elements with magnitude
# >= 5, I'm going to boolean index the magnitudes array
mask = mag_array >= 5

# I'm going to filter all arrays down with the mask
time_array, place_array, mag_array = (
    time_array[mask],
    place_array[mask],
    mag_array[mask],
)

# Then get an array of indices that would sort magnitude array from
# greatest to least
sort_indices = np.argsort(mag_array)[::-1]

# Then sort all arrays based on the magnitude array
time_array, place_array, mag_array = (
    time_array[sort_indices],
    place_array[sort_indices],
    mag_array[sort_indices],
)

In [123]:
from datetime import datetime

# Time in the dataset it given as a long integer, we want mm/dd/yyyy
# So we use python's convertion function, vectorized across the array
time_convert = np.vectorize(datetime.fromtimestamp)

# We get back an array of datetime objects, but we need strings
time_array = time_convert(time_array / 1000)


# Create a function to build a string from a datetime object
def to_date(a: datetime):
    return a.strftime("%m") + "/" + a.strftime("%d") + "/" + a.strftime("%Y")


# Vectorize the function
date_conversion = np.vectorize(to_date)

# apply it to our array
time_array = date_conversion(time_array)

In [ ]:
import pandas as pd

# We're going to bring our three arrays of data into one object
# to pass to pandas for our dataframe
data_object = np.column_stack((time_array.T, place_array.T, mag_array.T))

# Then we build the dataframe and label the columns
pd.DataFrame(
    data_object,
    columns=["Date", "Place", "Magnitude"],
)

,Date,Place,Magnitude
0,06/28/2025,Scotia Sea,6.6
1,06/23/2025,east of the Philippine Islands,6.2
2,06/25/2025,southern Mid-Atlantic Ridge,6.2
3,06/27/2025,"71 km ESE of Sarangani, Philippines",6.1
4,06/23/2025,"77 km NNE of Punta Cana, Dominican Republic",5.7
5,06/27/2025,"169 km NNW of Houma, Tonga",5.5
6,06/23/2025,"105 km SE of Kirakira, Solomon Islands",5.5
7,06/28/2025,"100 km NNW of Tatsugō, Japan",5.4
8,06/24/2025,"270 km SSE of Port Blair, India",5.4
9,06/23/2025,"31 km NNW of Asan-Maina Village, Guam",5.4
